<a href="https://colab.research.google.com/github/yk-Jeong/Codestates_Project/blob/main/N414_assignment_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving TelcomCustomer.csv to TelcomCustomer.csv


In [2]:
import pandas as pd
df = pd.read_csv('TelcomCustomer.csv')

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
#결측치 확인
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [5]:
#타입, 데이터 개형 확인
df.dtypes, df.shape

(customerID           object
 gender               object
 SeniorCitizen         int64
 Partner              object
 Dependents           object
 tenure                int64
 PhoneService         object
 MultipleLines        object
 InternetService      object
 OnlineSecurity       object
 OnlineBackup         object
 DeviceProtection     object
 TechSupport          object
 StreamingTV          object
 StreamingMovies      object
 Contract             object
 PaperlessBilling     object
 PaymentMethod        object
 MonthlyCharges      float64
 TotalCharges         object
 Churn                object
 dtype: object, (7043, 21))

In [6]:
#target column의 데이터 형태를 float으로 변경

df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors = 'coerce')

In [7]:
df = df.dropna()

In [8]:
df = df.drop(columns = 'customerID')

In [9]:
#data cleansing

no_internet_feats = ['TechSupport', 'OnlineBackup', 'DeviceProtection', 'StreamingTV', 'OnlineSecurity', 'StreamingMovies']
for i in no_internet_feats:
  df[i] = df[i].replace({'No internet service': 'No'})

df['MultipleLines'] = df['MultipleLines'].replace({'No phone service': 'No'})
df['SeniorCitizen'] = df['SeniorCitizen'].replace({0:'No', 1:'Yes'})

In [10]:
pip install category_encoders

In [11]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,No,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,No,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [12]:
#one-hot encoding

from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder()
df_encoded = encoder.fit_transform(df)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [13]:
df_encoded.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,29.85,29.85,1
1,2,1,2,1,34,2,1,1,2,2,2,1,1,1,2,2,2,56.95,1889.50,1
2,2,1,2,1,2,2,1,1,2,1,1,1,1,1,1,1,2,53.85,108.15,2
3,2,1,2,1,45,1,1,1,2,2,2,2,1,1,2,2,3,42.30,1840.75,1
4,1,1,2,1,2,2,1,2,1,2,1,1,1,1,1,1,1,70.70,151.65,2


In [14]:
#train/target 분리 
df_encoded['Churn'] = df_encoded['Churn'].replace({1:0, 2:1})
df_encoded['Churn'].value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [15]:
df_encoded.shape

(7032, 20)

Data Split: Train and test set

In [16]:
from sklearn.model_selection import train_test_split

(train, test) = train_test_split(df_encoded, test_size = 0.25, random_state=1)

In [17]:
train.shape, test.shape

((5274, 20), (1758, 20))

In [28]:
target = 'Churn'
features = df_encoded.drop(columns=[target]).columns

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

Scaling

In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [30]:
X_train_scaled[0]

array([-1.00952584, -0.44660293, -1.03866899, -0.65370807,  1.57743883,
        0.32362629,  1.15393457,  0.17745038, -0.63367288, -1.36841113,
        1.36726855, -0.64691929,  1.25757616,  1.26869981,  1.57728015,
       -0.82893915,  0.60478129,  1.33038512,  2.30352715])

Modeling

In [21]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

import keras
import tensorflow as tf
import IPython #얘는 무슨 역할을 하는가 

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [22]:
!pip install -U keras-tuner

In [23]:
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [31]:
np.unique(y_train)

array([0, 1])

In [32]:
tf.random.set_seed(7)

model_2 = Sequential()
model_2.add(Dense(64, activation = 'relu'))
model_2.add(Dense(64, activation = 'relu'))
model_2.add(Dense(1, activation = 'sigmoid'))

In [33]:
model_2.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
result = model_2.fit(X_train_scaled, y_train, epochs = 10, validation_data = (X_test_scaled, y_test))

Epoch 1/10
165/165 [==============================] - 2s 6ms/step - loss: 0.4696 - accuracy: 0.7643 - val_loss: 0.4214 - val_accuracy: 0.7975
Epoch 2/10
165/165 [==============================] - 1s 4ms/step - loss: 0.4240 - accuracy: 0.8032 - val_loss: 0.4191 - val_accuracy: 0.7941
Epoch 3/10
165/165 [==============================] - 1s 4ms/step - loss: 0.4165 - accuracy: 0.8030 - val_loss: 0.4181 - val_accuracy: 0.7975
Epoch 4/10
165/165 [==============================] - 1s 4ms/step - loss: 0.4118 - accuracy: 0.8110 - val_loss: 0.4191 - val_accuracy: 0.7958
Epoch 5/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4075 - accuracy: 0.8093 - val_loss: 0.4209 - val_accuracy: 0.7986
Epoch 6/10
165/165 [==============================] - 1s 4ms/step - loss: 0.4042 - accuracy: 0.8117 - val_loss: 0.4207 - val_accuracy: 0.7986
Epoch 7/10
165/165 [==============================] - 1s 4ms/step - loss: 0.4028 - accuracy: 0.8108 - val_loss: 0.4223 - val_accuracy: 0.7992
Epoch 

In [35]:
model_2.evaluate(X_test_scaled, y_test, verbose = 2)

55/55 - 0s - loss: 0.4230 - accuracy: 0.7952 - 112ms/epoch - 2ms/step


[0.42302030324935913, 0.7952218651771545]

GridSearchCV 

In [36]:
tf.random.set_seed(7)

def model_builder(nodes=16, activation='relu'):

  model = Sequential()
  model.add(Dense(nodes, activation=activation))
  model.add(Dense(nodes, activation=activation))
  model.add(Dense(1, activation='sigmoid')) #이진분류: 출력층의 node를 1로, activate func.는 sigmoid

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [37]:
model = KerasClassifier(build_fn = model_builder, verbose = 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [38]:
#hyperparameters

batch_size = [64, 128, 256]
epochs = [10, 20, 30]
nodes = [64, 128, 256]
activation = ['relu', 'sigmoid']

param_grid = dict(batch_size = batch_size, epochs = epochs, nodes = nodes, activation = activation)

In [39]:
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 3, verbose = 1, n_jobs = -1)
grid_result = grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


In [40]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params): #zip?! 신기한 함수다 
  print(f"Means: {mean}, Stdev: {stdev} \n with {param}")

Best: 0.8039438724517822 using {'activation': 'sigmoid', 'batch_size': 256, 'epochs': 30, 'nodes': 128}
Means: 0.7999620834986368, Stdev: 0.0036075794947286084 
 with {'activation': 'relu', 'batch_size': 64, 'epochs': 10, 'nodes': 64}
Means: 0.797686775525411, Stdev: 0.005732343495866289 
 with {'activation': 'relu', 'batch_size': 64, 'epochs': 10, 'nodes': 128}
Means: 0.7933257222175598, Stdev: 0.009117037682482553 
 with {'activation': 'relu', 'batch_size': 64, 'epochs': 10, 'nodes': 256}
Means: 0.7908608317375183, Stdev: 0.008181831831397979 
 with {'activation': 'relu', 'batch_size': 64, 'epochs': 20, 'nodes': 64}
Means: 0.7855517665545145, Stdev: 0.0056502475470928 
 with {'activation': 'relu', 'batch_size': 64, 'epochs': 20, 'nodes': 128}
Means: 0.7741751869519552, Stdev: 0.011271786883637148 
 with {'activation': 'relu', 'batch_size': 64, 'epochs': 20, 'nodes': 256}
Means: 0.7847933371861776, Stdev: 0.011236654224673457 
 with {'activation': 'relu', 'batch_size': 64, 'epochs': 3

Keras tunner case

In [45]:
def model_builder(hp):
  model = Sequential()

  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32) #입력층

  model.add(Dense(units = hp_units, activation = 'relu')) #은닉층
  model.add(Dense(units = hp_units, activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid')) #출력층

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
  
  model.compile(optimizer=keras.optimizers.Adam(learning_rate = hp_learning_rate), 
                loss = keras.losses.BinaryCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

In [47]:
from tensorflow import keras #이 lib 호출을 선언하지 않으면 위에 정의한 model_bulider에서 keras.optimizers.Adam을 못 찾는 사태가 발생^;

In [48]:
tuner = kt.Hyperband(model_builder, 
                     objective = 'val_accuracy', 
                     max_epochs = 30, 
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt'
                     )

출력 삭제 함수 

In [53]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*ars, **kwargs): #얘네가 뭘 의미하는지 아직 잘 모름  
    IPython.display.clear_output(wait = True)

In [54]:
tuner.search(X_train_scaled, y_train, epochs = 30, batch_size = 50, 
             validation_data = (X_test_scaled, y_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"최적화된 Dense 노드 수: {best_hps.get('units')}")
print(f"최적화된 learning rate: {best_hps.get('learning_rate')}")

Trial 63 Complete [00h 00m 03s]
val_accuracy: 0.8026165962219238

Best val_accuracy So Far: 0.806598424911499
Total elapsed time: 00h 03m 18s
INFO:tensorflow:Oracle triggered exit
최적화된 Dense 노드 수: 96
최적화된 learning rate: 0.001


In [58]:
from tensorflow.keras import regularizers

tf.random.set_seed(1442) #왜 하필 1442인가 
initializer = tf.keras.initializers.HeNormal()

model = Sequential()

model.add(Dense(best_hps.get('units'), 
                activation='relu', kernel_initializer=initializer, 
                kernel_regularizer=regularizers.l2(0.01), #L2 norm
                activity_regularizer = regularizers.l1(0.01)))

model.add(Dense(best_hps.get('units'), 
                activation='relu', kernel_initializer=initializer, 
                kernel_regularizer=regularizers.l2(0.01), #L2 norm
                activity_regularizer = regularizers.l1(0.01)))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = keras.optimizers.Adam(learning_rate = best_hps.get('learning_rate')), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [59]:
result = model.fit(X_train_scaled, y_train, epochs=2, batch_size=50, validation_data = (X_test_scaled, y_test))

Epoch 1/2
106/106 [==============================] - 2s 6ms/step - loss: 4.0118 - accuracy: 0.7655 - val_loss: 2.9816 - val_accuracy: 0.7850
Epoch 2/2
106/106 [==============================] - 0s 5ms/step - loss: 2.3525 - accuracy: 0.7916 - val_loss: 1.8246 - val_accuracy: 0.7873


In [60]:
model.evaluate(X_test_scaled, y_test, verbose = 2)

55/55 - 0s - loss: 1.8246 - accuracy: 0.7873 - 97ms/epoch - 2ms/step


[1.8246334791183472, 0.7872582674026489]